# Dataset Description
Dataset containing information regarding trains operating on the NJ Transit rail network between 2018-03-01 and 2020-05-18.

# Project Goals
- Stations with the most traffic (arrivals/departures)
- Time of day with the most train traffic
- Specific train ids with the highest chance of being cancelled
- Amtrak routes that make up most of the total cancellations

In [1]:
import pandas as pd
pd.set_option("mode.chained_assignment", None)
import os
import glob
import datetime as dt
import plotly_express as px
import html5lib

In [2]:
os.chdir("C:/Users/aleks/Jupyter Notebooks/amtrak_project")

#get all file names in directory
extension = "csv"
all_filenames = [i for i in glob.glob("*.{}".format(extension))]
print(all_filenames)

['2018_03.csv', '2018_04.csv', '2018_05.csv', '2018_06.csv', '2018_07.csv', '2018_08.csv', '2018_09.csv', '2018_11.csv', '2018_12.csv', '2019_01.csv', '2019_02.csv', '2019_03.csv', '2019_04.csv', '2019_05.csv', '2019_06.csv', '2019_07.csv', '2019_08.csv', '2019_09.csv', '2019_10.csv', '2019_11.csv', '2019_12.csv', '2020_01.csv', '2020_02.csv', '2020_03.csv', '2020_04.csv', '2020_05.csv', 'trains_combined.csv']


In [3]:
#concat the files from the list
combined_data = pd.concat([pd.read_csv(f) for f in all_filenames])

In [4]:
#print first 5 rows
print(combined_data.shape)
combined_data.iloc[:5]

(12189696, 13)


,date,train_id,stop_sequence,from,from_id,to,to_id,scheduled_time,actual_time,delay_minutes,status,line,type
0,2018-03-01,3805,1.0,New York Penn Station,105.0,New York Penn Station,105.0,2018-03-02 01:22:00,2018-03-02 01:21:05,0.000000,departed,Northeast Corrdr,NJ Transit
1,2018-03-01,3805,2.0,New York Penn Station,105.0,Secaucus Upper Lvl,38187.0,2018-03-02 01:31:00,2018-03-02 01:31:08,0.133333,departed,Northeast Corrdr,NJ Transit
2,2018-03-01,3805,3.0,Secaucus Upper Lvl,38187.0,Newark Penn Station,107.0,2018-03-02 01:40:00,2018-03-02 01:40:07,0.116667,departed,Northeast Corrdr,NJ Transit
3,2018-03-01,3805,4.0,Newark Penn Station,107.0,Newark Airport,37953.0,2018-03-02 01:45:00,2018-03-02 01:45:10,0.166667,departed,Northeast Corrdr,NJ Transit
4,2018-03-01,3805,5.0,Newark Airport,37953.0,North Elizabeth,109.0,2018-03-02 01:49:00,2018-03-02 01:49:10,0.166667,departed,Northeast Corrdr,NJ Transit


# Data Cleaning

In [5]:
#see what the different train types are
print(combined_data["type"].unique())

#see the different types of status a train can have
print(combined_data["status"].value_counts(), "\n")

#split into two separate dataframes for NJ transit and Amtrak
amtrak_combined = combined_data[combined_data["type"] == "Amtrak"]
nj_combined = combined_data[combined_data["type"] == "NJ Transit"]
print("amtrak:", amtrak_combined.shape)
print("nj:", nj_combined.shape)

['NJ Transit' 'Amtrak']
departed     10994184
estimated     1031356
cancelled      164156
Name: status, dtype: int64 

amtrak: (598074, 13)
nj: (11591622, 13)


In [6]:
#see the unique amtrak lines
print(amtrak_combined["line"].value_counts())

Amtrak                       194558
REGIONAL                     161204
ACELA EXPRESS                 96238
KEYSTONE                      78678
PENNSYLVANIAN                 11338
CRESCENT                      10610
AMTRAK                         9950
SILVER STAR  -R                9820
SILVER METEOR-R                9418
CAROLINIAN                     8136
VERMONTER    -R                4250
AMTRAK REGIONAL                1496
Palmetto                        716
Regional                        454
Raritan Valley                  204
PALMETTO                        196
Keystone                        138
Northeast Corrdr                118
Acela Express                   104
SILVER STAR -R                   96
Silver Service / Palmetto        82
VERMONTER -R                     40
No Jersey Coast                  38
Morristown Line                  32
Bergen Co. Line                  32
CARDINAL                         32
Silver Meteor                    30
Pennsylvanian               

In [7]:
#make line name lower case, remove "-r" and extra spaces
amtrak_combined["line"] = amtrak_combined["line"].str.lower()
amtrak_combined["line"] = amtrak_combined["line"].str.replace(r"-r", "")
amtrak_combined["train_id"] = amtrak_combined["train_id"].str.replace(",", "")
amtrak_combined["train_id"] = amtrak_combined["train_id"].str.replace("X", "")
amtrak_combined["line"] = amtrak_combined["line"].str.strip()

#drop unneeded columns
amtrak_combined = amtrak_combined[["date", "train_id", "from", "from_id", 
                                   "to", "to_id", "actual_time", "status", "line", "type"]]

#replace different names with original
amtrak_combined.replace(inplace=True, to_replace="regional", value="northeast regional")
amtrak_combined.replace(inplace=True, to_replace="northeast corrdr", value="northeast regional")
amtrak_combined.replace(inplace=True, to_replace="carolinian", value="carolinian / piedmont")
amtrak_combined.replace(inplace=True, to_replace="cardinal / hoosier state", value="cardinal")
amtrak_combined.replace(inplace=True, to_replace="amtrak regional", value="northeast regional")
amtrak_combined.replace(inplace=True, to_replace="palmetto", value="silver service / palmetto")

#convert id to int, date and actual_time to datetime
amtrak_combined["from_id"] = amtrak_combined["from_id"].astype(int)
amtrak_combined["to_id"] = amtrak_combined["to_id"].astype(int)
amtrak_combined["date"] = pd.to_datetime(amtrak_combined["date"])
amtrak_combined["actual_time"] = pd.to_datetime(amtrak_combined["actual_time"])
amtrak_combined["actual_time"] = pd.to_datetime(amtrak_combined["actual_time"], format="%Y-%m-%d %H:%M:%S")

#clean train id
str_train_id = (amtrak_combined["train_id"]
                .astype(str)
                .str.replace(".", ""))
str_train_id = str_train_id.str.replace("A", "")
str_train_id = str_train_id.str.replace("T", "")
amtrak_combined["train_id"] = str_train_id
amtrak_combined["train_id"] = amtrak_combined["train_id"].str.lstrip("0")

#create a new column with the hour
amtrak_combined["hour"] = amtrak_combined["actual_time"].dt.hour

#reset index
amtrak_combined.reset_index(drop=True, inplace=True)

amtrak_combined[:5]

C:\Users\aleks\AppData\Local\Temp\ipykernel_3644\2316577931.py:28: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  str_train_id = (amtrak_combined["train_id"]


,date,train_id,from,from_id,to,to_id,actual_time,status,line,type,hour
0,2018-03-01,186,Philadelphia,1,Philadelphia,1,2018-03-01 15:01:14,departed,northeast regional,Amtrak,15
1,2018-03-01,186,Philadelphia,1,Trenton,148,2018-03-01 15:33:13,departed,northeast regional,Amtrak,15
2,2018-03-01,186,Trenton,148,Metropark,83,2018-03-01 15:55:14,departed,northeast regional,Amtrak,15
3,2018-03-01,186,Metropark,83,Newark Airport,37953,2018-03-01 16:06:15,departed,northeast regional,Amtrak,16
4,2018-03-01,186,Newark Airport,37953,Newark Penn Station,107,2018-03-01 16:12:16,departed,northeast regional,Amtrak,16


In [8]:
#check for nulls
amtrak_len = amtrak_combined.shape[0]
no_na_amtrak_len = amtrak_combined.dropna().shape[0]

amtrak_len == no_na_amtrak_len

True

In [9]:
#see the different types of status an amtrak train can have
print(amtrak_combined["status"].value_counts())

cancelled = amtrak_combined["status"].value_counts()[1]
departed = amtrak_combined["status"].value_counts()[0]
print("\namtrak train cancellation percentage: {}%".format(round((cancelled/departed)*100, 2)))

departed     594918
cancelled      3156
Name: status, dtype: int64

amtrak train cancellation percentage: 0.53%


Amtrak trains in this dataset have an overall cancellation rate of 0.53%.

In [10]:
#remove train lines that are incorrectly categorized under the Amtrak type instead of NJ transit
amtrak_routes = pd.read_html("https://en.wikipedia.org/wiki/List_of_Amtrak_routes",
                            match="Route miles")
amtrak_routes = amtrak_routes[0]
amtrak_routes["Name"] = amtrak_routes["Name"].str.lower()
amtrak_routes = amtrak_routes["Name"]
amtrak_routes = amtrak_routes.tolist()


print(amtrak_combined["line"].value_counts(), "\n")
print(amtrak_routes)

not_amtrak = ["raritan valley", "no jersey coast", "bergen co. line", "morristown line"]


amtrak                       204508
northeast regional           163272
acela express                 96342
keystone                      78816
pennsylvanian                 11366
crescent                      10616
silver star                    9916
silver meteor                  9448
carolinian / piedmont          8160
vermonter                      4290
silver service / palmetto       994
raritan valley                  204
cardinal                         40
no jersey coast                  38
bergen co. line                  32
morristown line                  32
Name: line, dtype: int64 

['acela', 'adirondack', 'amtrak cascades', 'auto train', 'berkshire flyer', 'blue water', 'california zephyr', 'capitol corridor', 'capitol limited', 'cardinal', 'carolinian', 'city of new orleans', 'coast starlight', 'crescent', 'downeaster', 'empire builder', 'empire service', 'ethan allen express', 'hartford line', 'heartland flyer', 'hiawatha service', 'illini and saluki', 'illinois zephyra

In [11]:
#filter out the lines that are in the not amtrak list
amtrak_combined= amtrak_combined[amtrak_combined["line"].isin(not_amtrak) ==False]

print(amtrak_combined["line"].value_counts())

amtrak                       204508
northeast regional           163272
acela express                 96342
keystone                      78816
pennsylvanian                 11366
crescent                      10616
silver star                    9916
silver meteor                  9448
carolinian / piedmont          8160
vermonter                      4290
silver service / palmetto       994
cardinal                         40
Name: line, dtype: int64


# Analysis

In [12]:
#create a dataframe with the total departures and arrivals for each station
not_cancelled = amtrak_combined[amtrak_combined["status"] != "cancelled"]
departures_by_station = not_cancelled.groupby("from").size()
arrivals_by_station = not_cancelled.groupby("to").size()

frame = {"departures":departures_by_station,
        "arrivals":arrivals_by_station}

station_traffic = pd.DataFrame(frame)

station_traffic = station_traffic.fillna(0)
station_traffic["departures"] = station_traffic["departures"].astype(int)

#filter for stations with more than 100 departures
station_traffic = station_traffic.query("departures > 100")

In [13]:
#get date range for data
print((amtrak_combined["date"].min()), (amtrak_combined["date"].max()), sep="\n")

2018-03-01 00:00:00
2020-05-18 00:00:00


In [27]:
#graph total traffic by station
station_traffic_graph = px.bar(station_traffic,
                              title= "NJ Transit stations with the most Amtrak traffic<br><sup>2018-03-01 to 2020-05-18</sup>",
                              template="plotly_dark")
station_traffic_graph.update_layout(xaxis_title="STATION",
                                   yaxis_title="TOTAL ARRIVALS AND DEPARTURES")
station_traffic_graph.update_layout()

station_traffic_graph.show()

On the NJ Transit rail network, New York Penn Station, Newark Penn Station, and Philadelphia's 30th Street Station stand out as the stations with the highest amount of Amtrak departures/arrivals.

In [15]:
#create a dataframe with the total departures and arrivals by hour
departures_by_hour = not_cancelled.groupby("hour").size()
arrivals_by_hour = not_cancelled.groupby("hour").size()

frame = {"departures":departures_by_hour,
        "arrivals":arrivals_by_hour}

hourly_traffic = pd.DataFrame(frame)

hourly_traffic = hourly_traffic.fillna(0)
hourly_traffic["departures"] = hourly_traffic["departures"].astype(int)

hourly_traffic

,departures,arrivals
hour,,
0,5418,5418
1,3596,3596
2,248,248
3,3260,3260
4,4352,4352
5,9966,9966
6,19944,19944
7,24340,24340
8,31608,31608


In [28]:
#graph total traffic by hour
hourly_traffic_graph = px.bar(hourly_traffic,
                              title= "Amtrak traffic on the NJ Transit network, by hour<br><sup>2018-03-01 to 2020-05-18</sup>",
                              template="plotly_dark")
hourly_traffic_graph.update_layout(xaxis_title="HOUR",
                                   yaxis_title="TOTAL ARRIVALS AND DEPARTURES")
hourly_traffic_graph.update_layout()

hourly_traffic_graph.show()

Amtrak traffic peaks at around 3PM and is at its lowest at around 2AM. There are generally many more departures/arrivals during the day, as expected. Other notable times of high traffic are 11-12 AM and around 8 PM.

In [17]:
#identify train ids with high cancellation raters
id_count_amtraks = amtrak_combined["train_id"].value_counts()
cancelled_count_amtraks = amtrak_combined[amtrak_combined["status"] == "cancelled"]["train_id"].value_counts()
frame = {"scheduled departures":id_count_amtraks,"cancelled departures":cancelled_count_amtraks}

cancelled_of_total_amtraks = pd.DataFrame(frame)
cancelled_of_total_amtraks.fillna(0, inplace=True)
cancelled_of_total_amtraks["cancelled departures"] = (cancelled_of_total_amtraks["cancelled departures"].
                                                      astype(int))
cancelled_of_total_amtraks["cancellation percentage"] = (cancelled_of_total_amtraks["cancelled departures"]
                                                         / cancelled_of_total_amtraks["scheduled departures"]
                                                        )*100
cancelled_of_total_amtraks["cancellation percentage"] = round(
    cancelled_of_total_amtraks["cancellation percentage"], 2)

#identify the 10 train ids with the highest percentage of cancelled departures
top_10_cancellations= (cancelled_of_total_amtraks.sort_values("cancellation percentage", inplace=False, 
                                                              ascending=False).
                       head(10).
                      sort_index())

top_10_cancellations


,scheduled departures,cancelled departures,cancellation percentage
133,864,14,1.62
188,70,10,14.29
20,5064,82,1.62
71,948,12,1.27
79,6074,202,3.33
80,5838,176,3.01
91,5838,178,3.05
92,4510,106,2.35
97,5778,124,2.15
98,5352,146,2.73


In [29]:
#graph cancellation percentages by train id
id_cancellations_graph = px.bar(top_10_cancellations,
                                  x="cancellation percentage",
                                  y=top_10_cancellations.index,
                                  template="plotly_dark")
id_cancellations_graph.update_layout(title="Amtrak trains most likely to be cancelled on the NJ Transit network<br><sup>2018-03-01 to 2020-05-18</sup>", 
                                       xaxis_title="CANCELLATION PERCENTAGE",
                                       yaxis_title="AMTRAK TRAIN ID")
id_cancellations_graph.update_traces(marker_color="darkred")

id_cancellations_graph.show()

In [19]:
#identify Amtrak lines with high cancellation raters
line_count_amtraks = amtrak_combined["line"].value_counts()
cancelled_count_lines = amtrak_combined[amtrak_combined["status"] == "cancelled"]["line"].value_counts()
frame_lines = {"scheduled departures":line_count_amtraks,"cancelled departures":cancelled_count_lines}

cancelled_of_total_lines= pd.DataFrame(frame_lines)
cancelled_of_total_lines.fillna(0, inplace=True)
cancelled_of_total_lines["cancelled departures"] = (cancelled_of_total_lines["cancelled departures"].
                                                      astype(int))
cancelled_of_total_lines["cancellation percentage"] = (cancelled_of_total_lines["cancelled departures"]
                                                         / cancelled_of_total_lines["scheduled departures"]
                                                        )*100
cancelled_of_total_lines["cancellation percentage"] = round(
    cancelled_of_total_lines["cancellation percentage"], 2)
cancelled_of_total_lines

,scheduled departures,cancelled departures,cancellation percentage
acela express,96342,458,0.48
amtrak,204508,896,0.44
cardinal,40,0,0.00
carolinian / piedmont,8160,230,2.82
crescent,10616,134,1.26
keystone,78816,304,0.39
northeast regional,163272,528,0.32
pennsylvanian,11366,88,0.77
silver meteor,9448,224,2.37
silver service / palmetto,994,0,0.00


In [20]:
cancellations_pie = cancelled_of_total_lines.reset_index()
cancellations_pie = cancellations_pie.rename(columns={"index":"line"})

#there are many trains assigned only 'amtrak' as the line; removed these from the graph as
#there is no file to match train ids to a more accurate line cateogry
cancellations_pie = cancellations_pie[cancellations_pie["line"] != "amtrak"]
cancellations_pie = cancellations_pie[cancellations_pie["cancelled departures"] != 0]
cancellations_pie

,line,scheduled departures,cancelled departures,cancellation percentage
0,acela express,96342,458,0.48
3,carolinian / piedmont,8160,230,2.82
4,crescent,10616,134,1.26
5,keystone,78816,304,0.39
6,northeast regional,163272,528,0.32
7,pennsylvanian,11366,88,0.77
8,silver meteor,9448,224,2.37
10,silver star,9916,276,2.78
11,vermonter,4290,16,0.37


In [30]:
#graph cancellation percentages by train id
cancellations_pie_chart = px.pie(cancellations_pie, 
                                 values="cancelled departures",
                                 names="line",
                                title="Share of total Amtrak cancellations on NJ Transit network, by route<br><sup>2018-03-01 to 2020-05-18</sup>",
                                template="plotly_dark",
                                color_discrete_sequence=px.colors.sequential.RdBu)
cancellations_pie_chart.update_traces(textfont_size=20, marker=dict(line=dict(color='#000000', width=1)))
cancellations_pie_chart.show()

The northeast regional and acela express make up the majority of Amtrak cancellations on the NJ Transit network. This is to be expected as they have the most departures. The carolininan/piedmont line, on the other hand, makes up 10% of the total cancellations despite running much less frequently. It has the highest line-specific cancellation rate at 2.82%.